<a href="https://colab.research.google.com/github/Th33na/project-fakers/blob/main/project_fakers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import re
from urllib import request
import shutil
import gzip
import pandas as pd
import re
import language_tool_python
from google.colab import file
# import language_check

In [ ]:
from IPython.display import clear_output
try:
  !pip install hvplot
  !pip install holoviews
  !pip install language-check
  !pip install --upgrade language_tool_python
except:
  print("Error installing.")
finally:
  clear_output()
  print("Installed.")

Installed.


In [ ]:
TOOL = language_tool_python.LanguageTool('en-US')

INFO:language_tool_python.download_lt:Unzipping /tmp/tmp5n9gzvrt.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://www.languagetool.org/download/LanguageTool-5.7.zip to /root/.cache/language_tool_python.


In [ ]:
def get_mentions(review, wordsToSearchFor):
  for word in wordsToSearchFor:
    if re.search(word, review): return True
  return False

In [ ]:
def get_grammar_mistake_count(review, word_count = 0, threshold = 7):
  # of mistakes
  # using language-check
  if word_count >= threshold:
    matches = TOOL.check(review)
    return len(matches)
  return None

In [ ]:
review = "A sentence with a error in "
word_count = 6
get_grammar_mistake_count(review, word_count)

In [ ]:
review = "The the quick brow nfox"
get_grammar_mistake_count(review)

2

In [ ]:
review = "The quick brow nfox"
get_grammar_mistake_count(review)

In [ ]:
def get_word_count(review):
  # give the number of words in the review
  output = re.split("\s|(?<!\d)[.?!](?!\d)", review)
  if '' in output: output.remove('')
  return len(output)

In [ ]:
def download_data_to_csv(url):
    # download process
    print("downloading: ",url)
    file_title = re.split(pattern='/', string=url)[-1]
    urlrtv = request.urlretrieve(url=url, filename=file_title)
    
    # for ".tsv" to ".csv"
    title = re.split(pattern=r'\.tsv', string=file_title)[0] +".csv"
    
    # unzip ".gz" file
    with gzip.open(file_title, 'rb') as f_in:
        with open(title, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

In [ ]:
def download_json_data(url):
  # download process
  print("downloading json: ", url)
  file_title = re.split(pattern='/', string=url)[-1]
  urlrtv = request.urlretrieve(url=url, filename=file_title)

  title = file_title[0] +".json"

  data = []

  # unzip ".gz" file
  with gzip.open(file_title) as f_in:
    for l in f_in:
      data.append(json.loads(l.strip()))

  # convert list into pandas dataframe
  df = pd.DataFrame.from_dict(data)

  # write dataframe to .csv
  csv_title = file_title[0] + ".csv"
  df.to_csv(csv_title)     

In [ ]:
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"

download_data_to_csv(url)

downloading:  https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz


In [ ]:
# read ".csv" file for Amazon Reviews
reviews_df = pd.read_csv("amazon_reviews_us_Home_Entertainment_v1_00.csv",sep="\\t",engine = "python",na_values=["\\N"])


# display
reviews_df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,179886,RY01SAV7HZ8QO,B00NTI0CQ2,667358431,Aketek 1080P LED Protable Projector HD PC AV V...,Home Entertainment,4,0,0,N,Y,good enough for my purpose,not the best picture quality but surely suitab...,2015-08-31
1,US,37293769,R1XX8SDGJ4MZ4L,B00BUCLVZU,621695622,TiVo Mini with IR Remote (Old Version),Home Entertainment,5,0,0,N,N,Tell the Cable Company to Keep their Boxes!,Not only do my TiVo Minis replace the boxes th...,2015-08-31
2,US,8332121,R149Q3B5L33NN5,B00RBX9D5W,143071132,Apple TV MD199LL/A Bundle including remote and...,Home Entertainment,5,0,0,N,Y,Works perfectly!,Works perfectly! Very user friendly!,2015-08-31
3,US,47054962,R2ZVD69Z6KPJ4O,B00UJ3IULO,567816707,New Roku 3 6.5 Foot HDMI - Bundle - v1,Home Entertainment,1,0,2,N,Y,It doesn't work. Each time I try to use ...,"It doesn't work. Each time I try to use it, th...",2015-08-31
4,US,23413911,R1DIKG2G33ZLNP,B0037UCTXG,909557698,Generic DVI-I Dual-Link (M) to 15-Pin VGA (F) ...,Home Entertainment,4,0,0,N,Y,As pictured,I received the item pictured. I am unsure why...,2015-08-31


In [ ]:
# display column datatypes
reviews_df.dtypes

marketplace          object
customer_id           int64
review_id            object
product_id           object
product_parent        int64
product_title        object
product_category     object
star_rating           int64
helpful_votes         int64
total_votes           int64
vine                 object
verified_purchase    object
review_headline      object
review_body          object
review_date          object
dtype: object

In [ ]:
# retrieve only Roku reviews
roku_reviews_df = reviews_df[reviews_df["product_title"].str.contains("roku", na=False, flags=re.IGNORECASE)]

# display
roku_reviews_df

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
3,US,47054962,R2ZVD69Z6KPJ4O,B00UJ3IULO,567816707,New Roku 3 6.5 Foot HDMI - Bundle - v1,Home Entertainment,1,0,2,N,Y,It doesn't work. Each time I try to use ...,"It doesn't work. Each time I try to use it, th...",2015-08-31
13,US,2681147,RUQK5N4WH8UN8,B00FO12XY6,448806082,Roku HD Streaming Player (Certified Refurbished),Home Entertainment,5,0,0,N,Y,Five Stars,"Smooth and worked great, thanks!",2015-08-31
19,US,3004043,R3IIOLWHWC297U,B00MWCJ8VQ,946374680,Roku 3500XB Streaming Stick (HDMI) (Certified ...,Home Entertainment,5,0,0,N,Y,Five Stars,great product,2015-08-31
26,US,11268174,R21ULVCK88UZZZ,B00UJ3IREE,916687035,New Roku 2 6.5 Foot HDMI - Bundle - v1,Home Entertainment,5,0,1,N,Y,Five Stars,Love it,2015-08-31
27,US,46451109,R1OF4M9SVG6I29,B00UJ3IULO,567816707,New Roku 3 6.5 Foot HDMI - Bundle - v1,Home Entertainment,1,1,3,N,Y,I consider myself scammed,There was no indication this product is not me...,2015-08-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690528,US,51375658,R3JJRTEPH17DKI,B000127NL6,438258689,"Roku HD1000 ArtPack ""Nature"" for Roku HD1000 H...",Home Entertainment,5,1,1,N,Y,Great Artpact,I own several of the Roku art packs and this i...,2004-06-17
690736,US,35619109,R218E7LGCEP7U7,B0000DH8HH,912775848,Roku HD1000 High-Definition Digital Media Player,Home Entertainment,3,10,20,N,N,Second thoughts,This product is truly designed for owners of d...,2004-06-04
692466,US,53065024,RCNANI055M4ZO,B0000DH8HH,912775848,Roku HD1000 High-Definition Digital Media Player,Home Entertainment,4,19,23,N,N,"Very good product, although a bit too expensive",This is a very well executed product. <br />Af...,2004-02-12
693357,US,32635003,R2UZIGJVY141BS,B0000DH8HH,912775848,Roku HD1000 High-Definition Digital Media Player,Home Entertainment,4,37,40,N,N,Chris L,Have had mine now for a month. Works as advert...,2003-12-29


In [ ]:
# write dataframe to .csv
roku_reviews_df.to_csv("roku_reviews.csv")

In [ ]:
roku_wordcheck_df = roku_reviews_df[["review_headline", "review_body", "review_date"]]

roku_wordcheck_df

,review_headline,review_body,review_date
3,It doesn't work. Each time I try to use ...,"It doesn't work. Each time I try to use it, th...",2015-08-31
13,Five Stars,"Smooth and worked great, thanks!",2015-08-31
19,Five Stars,great product,2015-08-31
26,Five Stars,Love it,2015-08-31
27,I consider myself scammed,There was no indication this product is not me...,2015-08-31
...,...,...,...
690528,Great Artpact,I own several of the Roku art packs and this i...,2004-06-17
690736,Second thoughts,This product is truly designed for owners of d...,2004-06-04
692466,"Very good product, although a bit too expensive",This is a very well executed product. <br />Af...,2004-02-12
693357,Chris L,Have had mine now for a month. Works as advert...,2003-12-29


In [ ]:
roku_wordcheck_df["review_word_count"] = 0

# for index, row in roku_wordcheck_df.iterrows():
#   count = get_word_count(row["review_body"])
#   row["review_word_count"] = count
roku_wordcheck_df["review_word_count"] = roku_wordcheck_df.apply(lambda row: get_word_count(row["review_body"]), axis=1)
roku_wordcheck_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,review_headline,review_body,review_date,review_word_count
3,It doesn't work. Each time I try to use ...,"It doesn't work. Each time I try to use it, th...",2015-08-31,23
13,Five Stars,"Smooth and worked great, thanks!",2015-08-31,5
19,Five Stars,great product,2015-08-31,2
26,Five Stars,Love it,2015-08-31,2
27,I consider myself scammed,There was no indication this product is not me...,2015-08-31,31


In [ ]:
wordsToSearchFor = ['chromecast']
roku_wordcheck_df["mention"] = False

roku_wordcheck_df["mention"] = roku_wordcheck_df.apply(lambda row: get_mentions(row["review_body"].lower(), wordsToSearchFor), axis=1)
roku_wordcheck_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,review_headline,review_body,review_date,review_word_count,mention
3,It doesn't work. Each time I try to use ...,"It doesn't work. Each time I try to use it, th...",2015-08-31,23,False
13,Five Stars,"Smooth and worked great, thanks!",2015-08-31,5,False
19,Five Stars,great product,2015-08-31,2,False
26,Five Stars,Love it,2015-08-31,2,False
27,I consider myself scammed,There was no indication this product is not me...,2015-08-31,31,False


In [ ]:
roku_wordcheck_df.to_csv("roku_wordcheck_df.csv")

In [ ]:
word_threshold = 7
roku_wordcheck_df["grammer_check"] = 0

roku_wordcheck_df["grammer_check"] = roku_wordcheck_df.apply(lambda row: get_grammar_mistake_count(row["review_body"], row["review_word_count"] ), axis=1)
roku_wordcheck_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
file.download('roku_wordcheck_df.csv')

In [ ]:
roku_wordcheck_df.head()

NameError: ignored